# CBOW Implementation of Word2Vec

This is part of the background research that I'm working on for [viberary.pizza](https://viberary.pizza/).

## Background 

[Word2vec](https://arxiv.org/abs/1301.3781) was a critical point in NLP work, building on previous work in dimensionality reduction in NLP such as tf-idf, topic modeling, and latent semantic analysis to reduce vocabulary sizes for computational complexity, and additionally, to add context by embedding similar words in the same latent space. 

As of 2022, it's almost been superceded by [transformers-based architectures](https://e2eml.school/transformers.html), but it's still worth understanding how it works in a historical context, as well as because there is a fair amount of it [in production in Spark](https://spark.apache.org/docs/3.1.2/api/python/reference/api/pyspark.ml.feature.Word2Vec.html).  

# Word2Vec Implementation

There are numerous word2vec implementations in libraries like Spark and Tensorflow. There is not an exact one in PyTorch, but following[this code](https://github.com/FraLotito/pytorch-continuous-bag-of-words/blob/master/cbow.py), as well as reading about the [architecture here](https://towardsdatascience.com/word2vec-with-pytorch-implementing-original-paper-2cd7040120b0) and [here](https://jalammar.github.io/illustrated-word2vec/),  I was able to implement and understand how it works under te covers


Original explanation in [PyTorch implementation](https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html) is here. 

As a starting point, we take our raw data for input. 
Our [training set for Viberary is some sample the Goodreads input dataset](https://github.com/veekaybee/viberary#input-data-sample), 
which is a string of text containing the metadata for each unique book
id. 

For a single book id, it will contain the book description book title, etc. So a sample of a single book, will look like this


```
Raw text: All's Fairy in Love and War (Avalon: Web of Magic, #8) To Kara's astonishment, she discovers that a portal has opened in her bedroom closet and two goblins 
have fallen through! They refuse to return to the fairy realms and be drafted for an impending war. 
In an attempt to roust the pesky creatures, Kara falls through the portal, smack into the middle of a huge war.
Kara meets Queen Selinda, who appoints Kara as a Fairy Princess and assigns her an impossible task: 
to put an end to the war using her diplomatic skills.
```

This is initially stored as a Python string. 

Our final goal in learning a Word2Vec model with CBOW is, given an input phrase over a context window, to predict the word that's missing. The context window is how many words before and after the word we care about. So, given the phrase "Kara falls X the portal", we should be able to predict that the correct word is "through."

We do this in Word2Vec by continuously sampling from the raw text over the context window, where the context window around the word is the X variable and the word itself is the target variable. 

For the first example, "Kara falls the portal" is the context and "through" is the response variable. Then we shift the window by 1 word and generate another entry. This is the whole of the [continuous bag of words approach.](https://arxiv.org/pdf/1301.3781.pdf)

When we're first training the model, we pass these samples into the model and ask it to make a prediction on a single word given all these samples. The output is a vector of propabilities of the sample related to each word. We then compare that prediction to the actual label (I.e. for the sample "Kara falls X the portal" we KNOW the correct word is "through").

We compare the actual vector (i.e. where through = 1) to the probability vector, and the difference between the two is the loss. The parameters are passed to the model across multiple epochs and continuously updated until we minimize the loss, i.e. we get as close to the predicted word as possible. 

In the process of doing this prediction, we create a lookup table of words, or embeddings matrix, to their vector representations. It is these vectors that become our embeddings. Andiamo!

In [ ]:
!pip install -q torch  # if you don't have it already

In [ ]:
# we need these bois
import torch
import torch.nn as nn

## Text Preparation

In [ ]:
# First, we'll initialize our hyperparameters for the model:

CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right - this is our context window
EMBEDDING_DIM = 100  # size of the embeddings matrix - we'll get to this in a bit

In [ ]:
# Our tiny training dataset

raw_text = """To Kara's astonishment, she discovers that a portal has opened in her bedroom closet and two goblins have fallen through! They refuse to return to the fairy realms and be drafted for an impending war. 
In an attempt to roust the pesky creatures, Kara falls through the portal, 
smack into the middle of a huge war. Kara meets Queen Selinda, who appoints 
Kara as a Fairy Princess and assigns her an impossible task: 
to put an end to the war using her diplomatic skills.""".split()

In [ ]:
# Text preprocessing get only individual words
vocab = set(raw_text)  # dedup
vocab_size = len(vocab)

In [ ]:
print(vocab)

In [ ]:
# we create simple mappings of word to an index of the word
word_to_ix = {word: ix for ix, word in enumerate(vocab)}
ix_to_word = {ix: word for ix, word in enumerate(vocab)}

In [ ]:
print(word_to_ix)

In [ ]:
# Creating our training data and context window


def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)


data = []
for i in range(CONTEXT_SIZE, len(raw_text) - CONTEXT_SIZE):
    context = [raw_text[i - 2], raw_text[i - 1], raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target))

In [ ]:
# We have our [input, input, input, input, target]
# based on the context window of +2 words -2 words
# you can see how we're building words close to each other now
print(*data[0:10], sep="\n")

## Model Set Up
# CBOW Architecture 

We have two layers in the CBOW implementation of Word2Vec: an input Embedding layer that maps each word to a space in the embedding dictionary, a hidden linear activation layer, and then the output layer that is the proportional probabilities [softmax](https://en.wikipedia.org/wiki/Softmax_function) of all the correct words given an input window. 

The critical part is the first part, creating the Embeddings lookup. 

First, we associate each word in the vocabulary with an index, aka `{'she': 0, 'middle': 1, 'put': 2`

Then, what we want to do is create an embeddings table, or matrix, that we will multiply with these indices to map each one to its correct place in relation to the other indices via a table lookup, based on how many vectors you'd like to represent the word. 

There is a [really good explanation](https://stats.stackexchange.com/questions/270546/how-does-keras-embedding-layer-work/305032#305032) of how these are generated: 

``` 
For a given word, you create a one-hot vector based on its index and multiply it by the embeddings matrix, effectively replicating a lookup. For instance, for the word "soon" the index is 4, and the one-hot vector is [0, 0, 0, 0, 1, 0, 0]. If you multiply this (1, 7) matrix by the (7, 2) embeddings matrix you get the desired two-dimensional embedding, which in this case is [2.2, 1.4].
```

In [ ]:
# This way, when we create our second tower of book words, we know which ones are likely related to a given book